<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Athena_parameterized_queries_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Athena Queries:
-----------------

drop table if exists irisdemo;

create external table irisdemo (sepal_length double,sepal_width double,petal_length double,petal_width double,variety String)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
'separatorChar' = ',',
'quoteChar' = '"',
'escapeChar' = '\\'
)
STORED AS TEXTFILE
LOCATION 's3://irisseta/'  TBLPROPERTIES ("skip.header.line.count"="1");

select * from irisdemo;

select sum(sepal_length) from irisdemo where variety='Setosa';--250.29999999999998




select sum(sepal_length) from irisdemo where variety='Virginica';--329.3999999999999




select sum(sepal_length) from irisdemo where variety='Versicolor';--296.8

--to create the Prepared Statement

PREPARE iris_fulla FROM
SELECT sum(sepal_length)
FROM irisdemo
WHERE variety = ? ;

--to execute the Prepared Statement

EXECUTE iris_fulla USING 'Versicolor';

--to remove the prepared statement

DEALLOCATE PREPARE iris_fulla;





In [ ]:
!pip install boto3

In [ ]:
import boto3

In [ ]:
access_key=''
secret_key=''

In [ ]:
session = boto3.Session(
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,region_name='us-east-1'
)

In [ ]:
athena_client = session.client('athena')

In [ ]:
query_to_be_executed="{}"

In [ ]:
athena_query_response = athena_client.start_query_execution(
                              QueryString = query_to_be_executed,
                              QueryExecutionContext = {
                                  'Database': "{}"
                              },
                              ResultConfiguration = {
                                  'OutputLocation': "{}"
                              }
                            )
athena_query_execution_id = athena_query_response['QueryExecutionId']

while True:
   athena_query_execution_status = athena_client.get_query_execution(QueryExecutionId=athena_query_execution_id)
   athena_query_state = athena_query_execution_status['QueryExecution']['Status']['State']
   if athena_query_state=="QUEUED" or athena_query_state=="RUNNING":
     continue
   else:
     break
response = athena_client.get_query_results(
    QueryExecutionId=athena_query_execution_id
)

In [ ]:
response

In [ ]:
athena_query_execution_id